In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

In [2]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Convolution1D, MaxPooling1D
from keras.datasets import imdb
from keras import backend as K

Using TensorFlow backend.


## General parameters

In [3]:
max_features = 5000  # vocabulary size
maxlen = 400         # max document length 
batch_size = 32      # minibatch size 

## CNN Parameters

In [21]:
# set parameters:
embedding_dims = 200 # size of embedding dims
nb_filter = 250      # number of filters 
filter_length = 3    # 1d convolution size
hidden_dims = 250    # size of hidden layers 
nb_epoch = 2         # 

## Load Data

In [5]:
print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

Loading data...
65560576/65552540 [==============================] - 1s     
25000 train sequences
25000 test sequences


In [6]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
X_train shape: (25000, 400)
X_test shape: (25000, 400)


In [22]:

def build_cnn(embedding_dims, maxlen, nb_filter, filter_length, hidden_dims ) : 
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen,
                    dropout=0.2))

    # we add a Convolution1D, which will learn nb_filter
    # word group filters of size filter_length:
    model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
    # we use max pooling:
    model.add(MaxPooling1D(pool_length=model.output_shape[1]))

    # We flatten the output of the conv layer,
    # so that we can add a vanilla dense layer:
    model.add(Flatten())

    # We add a vanilla hidden layer:
    model.add(Dense(hidden_dims))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))

    # We project onto a single unit output layer, and squash it with a sigmoid:
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model

In [23]:
print('Build model...')
model = build_cnn(embedding_dims, maxlen, nb_filter, filter_length, hidden_dims )

Build model...


In [24]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_4 (Embedding)          (None, 400, 200)      1000000     embedding_input_4[0][0]          
____________________________________________________________________________________________________
convolution1d_4 (Convolution1D)  (None, 398, 250)      150250      embedding_4[0][0]                
____________________________________________________________________________________________________
maxpooling1d_4 (MaxPooling1D)    (None, 1, 250)        0           convolution1d_4[0][0]            
____________________________________________________________________________________________________
flatten_4 (Flatten)              (None, 250)           0           maxpooling1d_4[0][0]             
___________________________________________________________________________________________

In [25]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(X_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 109s - loss: 0.4054 - acc: 0.8076 - val_loss: 0.2881 - val_acc: 0.8812
Epoch 2/2
17408/25000 [===================>..........] - ETA: 27s - loss: 0.2668 - acc: 0.8908